## 딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회

**※주의** : 반드시 본 파일을 이용하여 제출을 수행해야 하며, 파일의 이름은 `task.ipynb`로 유지되어야 합니다.

* #### 추론 실행 환경
    * `python 3.9` 환경
    * `CUDA 10.2`, `CUDA 11.8`, `CUDA 12.6`를 지원합니다.
    * 각 CUDA 환경에 미리 설치돼있는 torch 버전은 다음 표를 참고하세요.

<table>
  <thead>
    <tr>
      <th align="center">Python</th>
      <th align="center">CUDA</th>
      <th align="center">torch</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="center" style="vertical-align: middle;">3.8</td>
      <td align="center">10.2</td>
      <td align="center">1.6.0</td>
    </tr>
    <tr>
      <td align="center" style="vertical-align: middle;">3.9</td>
      <td align="center">11.8</td>
      <td align="center">1.8.0</td>
    </tr>
    <tr>
      <td align="center">3.10</td>
      <td align="center">12.6</td>
      <td align="center">2.7.1</td>
    </tr>
  </tbody>
</table>

* #### CUDA 버전 관련 안내사항  
  - 이번 경진대회는 3개의 CUDA 버전을 지원합니다.  
  - 참가자는 자신의 모델의 라이브러리 의존성에 맞는 CUDA 환경을 선택하여 모델을 제출하면 됩니다.   
  - 각 CUDA 환경에는 기본적으로 torch가 설치되어 있으나, 참가자는 제출하는 CUDA 버전과 호환되는 torch, 필요한 버전의 라이브러리를 `!pip install` 하여 사용하여도 무관합니다.

* #### `task.ipynb` 작성 규칙
코드는 크게 3가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하세요.   
1. **제출용 aifactory 라이브러리 및 추가 필요 라이브러리 설치**
    - 채점 및 제출을 위한 aifactory 라이브러리를 설치하는 셀입니다. 이 부분은 수정하지 않고 그대로 실행합니다.
    - 그 외로, 모델 추론에 필요한 라이브러리를 직접 설치합니다.
2. **추론용 코드 작성**
    - 모델 로드, 데이터 전처리, 예측 등 실제 추론을 수행하는 모든 코드를 이 영역에 작성합니다.
3. **aif.submit() 함수를 호출하여 최종 결과를 제출**
    - **마이 페이지-활동히스토리**에서 발급받은 key 값을 함수의 인자로 정확히 입력해야 합니다.
    - **※주의** : 제출하고자 하는 CUDA 환경에 맞는 key를 입력하여야 합니다.

<table>
  <thead>
    <tr>
      <th align="left">Competition 이름</th>
      <th align="center">CUDA</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회</td>
      <td align="center">11.8</td>
    </tr>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회 CUDA 12.6</td>
      <td align="center">12.6</td>
    </tr>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회 CUDA 10.2</td>
      <td align="center">10.2</td>
    </tr>
  </tbody>
</table>

------

#### 1. 제출용 aifactory 라이브러리 설치
※ 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [1]:
!pip install -U aifactory

* 자신의 모델 추론 실행에 필요한 추가 라이브러리 설치

In [ ]:
!pip install transformers==4.30
!pip install -U torch==2.7.1 torchvision==0.22.1 --index-url https://download.pytorch.org/whl/cu118
!pip install -U datasets
!pip install -U opencv-python==4.10.0.82 numpy==1.26.4 scikit-learn==1.3.2 scipy==1.11.4
!pip install -U pathlib
!pip install -U dlib

-----

#### 2. 추론용 코드 작성

##### 추론 환경의 기본 경로 구조

- 평가 데이터셋 경로: `./data/`
   - 채점에 사용될 테스트 데이터셋은 `./data/` 디렉토리 안에 포함되어 있습니다.
   - 해당 디렉토리에는 이미지(JPG, PNG)와 동영상(MP4) 파일이 별도의 하위 폴더 없이 혼합되어 있습니다.
```bash
/aif/
└── data/
    ├── {이미지 데이터1}.jpg
    ├── {이미지 데이터2}.png
    ├── {동영상 데이터1}.mp4
    ├── {이미지 데이터3}.png
    ├── {동영상 데이터2}.mp4
    ...
```

- 모델 및 자원 경로: 예시 : `./model/`
   - 추론 스크립트가 실행되는 위치를 기준으로, 제출된 모델 관련 파일들이 위치해야하 하는 상대 경로입니다.
   - 학습된 모델 가중치(.pt, .ckpt, .pth 등)

* 제출 파일은 `submission.csv`로 저장돼야 합니다.
  * submission.csv는 *filename*과 *label* 컬럼으로 구성돼야 합니다.
  * filename은 추론한 파일의 이름(확장자 포함), label은 추론 결과입니다. (real:0, fake:1)
  * filename은 *string*, label은 *int* 자료형이어야 합니다.
  * 추론하는 데이터의 순서는 무작위로 섞여도 상관 없습니다.

<table>
  <thead>
    <tr>
      <th align="center">filename</th>
      <th align="center">label</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="center">{이미지 데이터1}.jpg</td>
      <td align="center">0</td>
    </tr>
    <tr>
      <td align="center">{동영상 데이터1}.mp4</td>
      <td align="center">1</td>
    </tr>
    <tr>
      <td colspan="2" align="center">...</td>
    </tr>
  </tbody>
</table>

**※ 주의 사항**

* argparse 사용시 `args, _ = parser.parse_known_args()`로 인자를 지정하세요.   
   - `args = parser.parse_args()`는 jupyter에서 오류가 발생합니다.
* return 할 결과물과 양식에 유의하세요.

In [ ]:
## 필요 라이브러리 import
import os
from transformers import ViTForImageClassification, ViTImageProcessor
from PIL import Image
import cv2
import dlib
from pathlib import Path
import numpy as np
import csv
import torch
import torch.nn.functional as F
from tqdm import tqdm
import multiprocessing # 병렬 처리를 위해 추가

### 추론 환경 경로 설정
# EXP-004: FaceForensics++ 파인튜닝 모델 사용
# Colab에서 "wilddeepfake_finetuned" 이름으로 저장됨
model_path = "./model/wilddeepfake_finetuned"
test_dataset_path = Path("./data")
output_csv_path = Path("submission.csv")

# --- 유틸리티 함수 및 상수 (기존과 동일) ---
IMAGE_EXTS = {".jpg", ".jpeg", ".png"}
VIDEO_EXTS = {".avi", ".mp4"}

def get_boundingbox(face, width, height):
    x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
    size_bb = int(max(x2 - x1, y2 - y1) * 1.3)
    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
    x1 = max(int(center_x - size_bb // 2), 0)
    y1 = max(int(center_y - size_bb // 2), 0)
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)
    return x1, y1, size_bb

def detect_and_crop_face_optimized(image: Image.Image, target_size=(224, 224), resize_for_detection=640):
    if image.mode != 'RGB': image = image.convert('RGB')
    original_np = np.array(image)
    original_h, original_w, _ = original_np.shape
    if original_w > resize_for_detection:
        scale = resize_for_detection / float(original_w)
        resized_h = int(original_h * scale)
        resized_np = cv2.resize(original_np, (resize_for_detection, resized_h), interpolation=cv2.INTER_AREA)
    else:
        scale = 1.0
        resized_np = original_np
    
    # dlib detector는 전역 변수로 사용하지 않고, 각 프로세스에서 새로 생성하는 것이 안전합니다.
    face_detector = dlib.get_frontal_face_detector()
    faces = face_detector(resized_np, 1)

    if not faces: return None
    face = max(faces, key=lambda rect: rect.width() * rect.height())
    scaled_face_rect = dlib.rectangle(
        left=int(face.left() / scale), top=int(face.top() / scale),
        right=int(face.right() / scale), bottom=int(face.bottom() / scale)
    )
    x, y, size = get_boundingbox(scaled_face_rect, original_w, original_h)
    cropped_np = original_np[y:y + size, x:x + size]
    face_img = Image.fromarray(cropped_np).resize(target_size, Image.BICUBIC)
    return face_img

# ####################################################################
# # 병렬 처리를 위한 함수: 파일 하나를 전처리하는 역할
# ####################################################################
def process_single_file(file_path):
    """파일 경로를 입력받아 전처리된 얼굴 이미지 리스트와 파일 이름을 반환"""
    face_images = []
    ext = file_path.suffix.lower()
    num_frames_to_extract = 40  # EXP-002: 30 → 40 (더 많은 프레임 샘플링)

    try:
        if ext in IMAGE_EXTS:
            image = Image.open(file_path)
            face_img = detect_and_crop_face_optimized(image)
            if face_img:
                face_images.append(face_img)
        elif ext in VIDEO_EXTS:
            cap = cv2.VideoCapture(str(file_path))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if total_frames > 0:
                frame_indices = np.linspace(0, total_frames - 1, num_frames_to_extract, dtype=int)
                for idx in frame_indices:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                    ret, frame = cap.read()
                    if not ret: continue
                    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                    face_img = detect_and_crop_face_optimized(image)
                    if face_img:
                        face_images.append(face_img)
            cap.release()
    except Exception as e:
        # 오류가 발생한 파일 경로와 오류 메시지를 반환하여 디버깅에 활용할 수 있습니다.
        return file_path.name, [], str(e)

    return file_path.name, face_images, None

# --- 메인 처리 로직 ---
if __name__ == "__main__":
    # 모델, 프로세서 로드
    model = ViTForImageClassification.from_pretrained(model_path).to("cuda")
    processor = ViTImageProcessor.from_pretrained(model_path)
    model.eval()
    print("model successfully loaded.")

    # CSV 파일 헤더 작성
    with open(output_csv_path, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "label"])

    files = [p for p in sorted(test_dataset_path.iterdir()) if p.is_file()]
    print("Test data length:", len(files))

    # 사용할 CPU 코어 수 설정 (전체 코어 - 1개를 하여 시스템 여유분 확보 / 최대 8개로 억제)
    num_workers = min(max(1, multiprocessing.cpu_count() - 1), 8)
    print(f"Using {num_workers} worker processes for preprocessing.")

    # 결과를 저장할 딕셔너리
    results_to_write = {}

    # multiprocessing.Pool을 사용하여 파일 전처리를 병렬로 실행
    with multiprocessing.Pool(processes=num_workers) as pool:
        # imap을 사용하면 결과를 받는 즉시 처리 가능, tqdm으로 진행률 표시
        with tqdm(total=len(files), desc="Preprocessing files") as pbar:
            for filename, face_images, error in pool.imap_unordered(process_single_file, files):
                if error:
                    print(f"Error processing {filename}: {error}")
                
                if not face_images:
                    # 전처리 실패 또는 얼굴 미검출 시 레이블 0으로 저장
                    results_to_write[filename] = 0
                else:
                    # EXP-003-Fixed: TTA (Test Time Augmentation) - 메모리 최적화
                    # 원본과 반전을 분리하여 추론 (메모리 폭발 방지)
                    
                    # 1. 원본 이미지 추론
                    inputs_orig = processor(images=face_images, return_tensors="pt").to("cuda")
                    with torch.no_grad():
                        outputs_orig = model(**inputs_orig)
                        probs_orig = F.softmax(outputs_orig.logits, dim=1).mean(dim=0)
                    
                    # 2. 좌우 반전 이미지 추론
                    flipped_images = [img.transpose(Image.FLIP_LEFT_RIGHT) for img in face_images]
                    inputs_flip = processor(images=flipped_images, return_tensors="pt").to("cuda")
                    with torch.no_grad():
                        outputs_flip = model(**inputs_flip)
                        probs_flip = F.softmax(outputs_flip.logits, dim=1).mean(dim=0)
                    
                    # 3. TTA 앙상블 (원본 + 반전 평균)
                    avg_probs = (probs_orig + probs_flip) / 2.0
                    predicted_class = torch.argmax(avg_probs).item()
                    results_to_write[filename] = predicted_class
                
                pbar.update(1)

    # 모든 추론이 끝난 후, 파일 이름 순서대로 정렬하여 CSV에 한 번에 기록
    print("Writing results to CSV...")
    with open(output_csv_path, mode="a", newline="") as f:
        writer = csv.writer(f)
        for p in files:
            filename = p.name
            label = results_to_write.get(filename, 0) # 만약 결과가 없으면 0으로 처리
            writer.writerow([filename, label])

    print("Inference completed.")

ModuleNotFoundError: No module named 'dlib'

----

#### 3. `aif.submit()` 함수를 호출하여 최종 결과를 제출

**※주의** : task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 대회 페이지 [베이스라인 코드](https://aifactory.space/task/9197/baseline) 탭에 기재된 가이드라인을 따라 task 별로 확인하실 수 있습니다.
- key가 틀리면 제출이 진행되지 않거나 잘못 제출되므로 task에 맞는 자신의 key를 사용해야 합니다.
-  **NOTE** : 이번 경진대회에서는 3개의 CUDA 버전을 지원하며, 각 CUDA 버전에 따라 task key가 상이합니다. 함수를 실행하기 전에 현재 key가 제출하고자 하는 CUDA 환경에 대한 key인지 반드시 확인하세요.

In [ ]:
import aifactory.score as aif
import time
t = time.time()

#-----------------------------------------------------#
# EXP-004: FaceForensics++ 파인튜닝 모델 + TTA + CUDA 12.6
aif.submit(model_name="EXP-004-FF++-Finetuned-TTA",
    key="492f7ace-67d6-4636-8448-25def840e236"
)
#-----------------------------------------------------#
print(time.time() - t)

file : task
jupyter notebook
제출 완료
188.92947268486023
